In [ ]:
# notebook settings
%load_ext autoreload
%autoreload 2
# %matplotlib notebook

# external imports
import numpy as np
import sympy as sp
from scipy.linalg import solve_discrete_are
import matplotlib.pyplot as plt

# internal imports
from dynamics import mld, x_max
from warm_start_hmpc.controller import HybridModelPredictiveController

In [ ]:
# controller horizon
T = 40

# weight matrices
C = np.eye(mld.nx)
D = np.vstack([1.]+[0.]*(mld.nu - 1)).T

# terminal cost
Bu = mld.B[:,0:1]
Du = D[:,0:1]
P = solve_discrete_are(mld.A, Bu, C.T.dot(C), Du.T.dot(Du))
C_T = np.linalg.cholesky(P).T
C_T = C
objective = [C, D, C_T]

# terminal constraints
F_T = np.vstack((np.eye(mld.nx), - np.eye(mld.nx)))
# h_T = np.ones(2*mld.nx) * .2
h_T = np.concatenate((x_max, x_max))
terminal_set = [F_T, h_T]

# hybrid controller
controller = HybridModelPredictiveController(mld, T, objective, terminal_set)

# initial push towards the right wall
x0 = np.array([0., 0., 1., 0.])

## Open-loop solution

In [ ]:
# solve in open loop
solution, leaves = controller.feedforward(
    x0,
    draw_label='Cart pole with walls'
)

## Solve with and without warm start

In [ ]:
e0 = np.random.randn(mld.nx)*0.
uc0 = solution.variables['uc'][0]
ub0 = solution.variables['ub'][0]
warm_start = controller.construct_warm_start(leaves, x0, uc0, ub0, e0)

In [ ]:
u0 = np.concatenate((uc0, ub0))
x1 = mld.A.dot(x0) + mld.B.dot(u0) + e0
solution, leaves = controller.feedforward(
    x1,
    warm_start=warm_start,
    draw_label='Cart pole with walls warm start'
)

In [ ]:
solution, leaves = controller.feedforward(x1)

## Solve in closed loop

In [ ]:
np.random.seed(1)

T_sim = 50
x_sim = [x0]
u_sim = []
warm_start = None

for t in range(T_sim):
    print('Time step %d.'%t)
    
    solution, leaves = controller.feedforward(
        x_sim[-1],
        warm_start=warm_start,
        printing_period=5.
    )
    
    [x0, x1] = solution.variables['x'][0:2]
    uc0 = solution.variables['uc'][0]
    ub0 = solution.variables['ub'][0]
    e0 = np.random.randn(mld.nx)*0.
    
    warm_start = controller.construct_warm_start(leaves, x0, uc0, ub0, e0)
    
    x_sim.append(x1)
    u_sim.append(uc0)

## Animation

In [ ]:
from visualizer import vis, animate
vis.jupyter_cell()

In [ ]:
animate(x_sim)